# Lab 2022-04-22: Satisfiability Modulo Theories

During this lab you will be using Z3 solver to deduce facts and solve combinatorial problems.

There are two ways to use Z3, either via the [SMT-LIB](https://jfmc.github.io/z3-play/) language or via a [programmatic APIs](https://z3prover.github.io/api/html/index.html). First we'll see an example of the SMT-LIB interface, but we'll focus on the Python API. The material covered in the [Z3Py tutorial](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) is enough for our purposes. For more examples and advanced usage you can look at the [SAT/SMT by Example](https://sat-smt.codes/SAT_SMT_by_example.pdf) book.


## Using Z3 in Jupyter

As an example we can use the solver to verify that, within the boolean logic, if $p\Rightarrow q$ and $q\Rightarrow r$, then $p\Rightarrow r$ (transitivity of implication). This can be verified by showing that the formula $$((p\Rightarrow q) \land (q\Rightarrow r)) \Rightarrow (p\Rightarrow r)$$ is a tautology: for any value True/False assigned to $ p, q , r$ the formula is true; or by checking that its negation $$\neg(((p\Rightarrow q) \land (q\Rightarrow r)) \Rightarrow (p\Rightarrow r))$$ cannot be satisfied.




### SMT-LIB

Code cells are executed by the selected kernel (Python 3, based on [IPython](https://ipython.readthedocs.io/en/stable/)), which provides [Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to a better interaction with the environment. In particular we'll use `%%script` command which works like the [Unix Shebang](https://en.wikipedia.org/wiki/Shebang_(Unix)) passing the content of the cell to the executing program as standard input.

In [1]:
%%script z3 -in -smt2

(declare-const p Bool)
(declare-const q Bool)
(declare-const r Bool)
(assert (not (=> (and (=> p q) (=> q r))
                 (=> p r))))
(check-sat)

unsat


The converse is not true, that is $$(p\Rightarrow r) \Rightarrow ((p\Rightarrow q) \land (q\Rightarrow r))$$ is not a tautology:

In [2]:
%%script z3 -in -smt2

(declare-const p Bool)
(declare-const q Bool)
(declare-const r Bool)
(assert (not (=> (=> p r)
                 (and (=> p q) (=> q r)))))
(check-sat)

sat


### Python API

Let's see the same example using the Python API:

In [3]:
from z3 import *

p, q, r = Bools('p q r')

s = Solver()

s.add(Not(Implies(And(Implies(p, q), Implies(q, r)), Implies(p, r))))
print(s.check())


unsat


Note that here we import the whole `z3` namespace, which is fine for small examples; but in general it might clash with names defined in your code.

Let's see the converse

In [4]:
from z3 import *

p, q, r = Bools('p q r')

s = Solver()

s.add(Not(Implies(Implies(p, r), And(Implies(p, q), Implies(q, r)))))
print(s.check())

sat


since the negation of the formula is satisfiable, it's not a tautology. We can look at the model for a counterexample:

In [5]:
print(f'A counterexample: {s.model()}')
for v in [p, q, r]:
    print(f'The value of {v} is {s.model()[v]}')


A counterexample: [p = True, q = False, r = True]
The value of p is True
The value of q is False
The value of r is True


## Z3 as SAT solver

Z3 can be used to solve Boolean Satisfiability (SAT) problems. 



### Unicorn example

Consider the unicorn example introduced during the lecture:

-   If the unicorn is mythical, then it is immortal
-   If the unicorn is not mythical, then it is a mortal mammal
-   If the unicorn is either immortal or a mammal, then it is horned
-   The unicorn is magical if it is horned

Abstracting the domain using propositional variables

-   $m$: the unicorn is mythical
-   $i$: the unicorn is immortal
-   $l$: the unicorn is mammal
-   $h$: the unicorn is horned
-   $g$: the unicorn is magical

the above statements became

$$(m\rightarrow i)\wedge(\neg m \rightarrow (\neg i \wedge l))\wedge((i \vee l) \rightarrow h)\wedge(h\rightarrow g)$$

The corresponding assertions are:

In [41]:
from z3 import *

m, i, l, h, g = Bools('m i l h g')

s = Solver()

s.add(Implies(m, i))
s.add(Implies(Not(m), And(Not(i), l)))
s.add(Implies(Or(i, l), h))
s.add(Implies(h, g))



We can check that our assumptions are not contradictory:

In [17]:
s.check()
#s.model()

sat

Now we can answer some questions about the domain.

1.  The unicorn must be mythical?

To answer that we use a statement that negate the question, but that's just an assumption that we use to verify the property and then retract. To this end we can use the `push`, `pop` methods of the solver (more details in [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-scopes)). The first creates a nested context and the second deletes it by retracting all assertions added after the push. So let's check whether the unicorn cannot be mythical (e.g., $m$ can be false):

In [43]:

print(s.assertions())

s.push()
s.add(Not(m))
print(s.assertions())
print(s.check())
s.pop()


s.push()
s.add(m)
print(s.assertions())
print(s.check())
print(s.model())
s.pop()

print(s.assertions())



[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 Not(m)]
sat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 m]
sat
[i = True, h = True, g = True, m = True]
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]


As you see $m$ can be both true or false, so we cannot tell anything about the unicorn being mythical. Now verify the other properties:

2.  Must be magical?

In [14]:
print(s.assertions())
s.push()
s.add(Not(g))
print(s.assertions())
print(s.check())
s.pop()
print(s.assertions())

[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 Not(g)]
unsat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]


3.  Must be horned?

In [18]:
print(s.assertions())
s.push()
s.add(Not(h))
print(s.assertions())
print(s.check())
s.pop()
print(s.assertions())

[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 Not(h)]
unsat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]


### Graph colouring

In the previous exercise we looked at SAT as a way to answer to questions; i.e. by focusing on logical deduction. However SAT can be used to find solutions to combinatorial (NP) problems. In this exercise we will consider the [Graph Colouring problem](https://en.wikipedia.org/wiki/Graph_coloring).

As seen in the lecture Graph Colouring can be used to solve different problems (e.g. [scheduling](https://en.wikipedia.org/wiki/Graph_coloring#Scheduling)).

Consider the following problem by James L. Hein:

> Some people form committees to do various tasks. The problem is to schedule the committee meetings in as few time slots as possible.
> We'll represent each person with a number. For example, let $S = \{1, 2, 3, 4, 5, 6, 7\}$ represent a set of seven people, and suppose they have formed six three-person committees as follows:
>
> $$S_1 = \{1, 2, 3\}, S_2 = \{2, 3, 4\}, S_3 = \{3, 4, 5\}, S_4 = \{4, 5, 6\}, S_5 = \{5, 6, 7\}, S_6 = \{1, 6, 7\}$$
>
> We can model the problem with a graph, where the committee names are the vertices and an edge connects two vertices if a person belongs to both committees represented by the vertices.
> If each committee meets for one hour, what is the smallest number of hours needed for the committees to do their work?
> From the graph, it follows that an edge between two committees means that they have at least one member in common. Thus, they cannot meet at the same time. No edge between committees means that they can meet at the same time. For example, committees $S_1$ and $S_4$ can meet at the same hour.

Solve the problem using a graph colouring problem with Z3. Note that variables can be also created by means of Python comprehension:

```python
V = tuple(Bool(f's_{i}') for i in range(7))

s = Solver()
s.add(Or(V[0], V[1]))
s.assertions()
```

note the `Bool` method (without the `s` at the end).

In [2]:
from z3 import *

def getValueFromList(V, key):
    desired_dict = next(item for item in V if item['key'] == key)

    # Extract the value corresponding to the key
    return desired_dict['value']

#Define terms
V=[]
for i in range(1,7): 
    for j in range(1,7):
        V.append({'key':f's_{i}_{j}', 'value':Bool (f's_{i}_{j}')})

s=Solver()
#Define conflicts between committee
#s_i_j is an interpretation where i is the committee and j is the assigned timeslot
for i in range(1,7):
    s.add(Implies(getValueFromList(V,f's_1_{i}'), Not(getValueFromList(V,f's_2_{i}'))))
    s.add(Implies(getValueFromList(V,f's_1_{i}'), Not(getValueFromList(V,f's_3_{i}'))))
    s.add(Implies(getValueFromList(V,f's_1_{i}'), Not(getValueFromList(V,f's_6_{i}'))))
    s.add(Implies(getValueFromList(V,f's_2_{i}'), Not(getValueFromList(V,f's_3_{i}'))))
    s.add(Implies(getValueFromList(V,f's_2_{i}'), Not(getValueFromList(V,f's_4_{i}'))))
    s.add(Implies(getValueFromList(V,f's_3_{i}'), Not(getValueFromList(V,f's_4_{i}'))))
    s.add(Implies(getValueFromList(V,f's_3_{i}'), Not(getValueFromList(V,f's_5_{i}'))))
    s.add(Implies(getValueFromList(V,f's_4_{i}'), Not(getValueFromList(V,f's_5_{i}'))))
    s.add(Implies(getValueFromList(V,f's_4_{i}'), Not(getValueFromList(V,f's_6_{i}'))))
    s.add(Implies(getValueFromList(V,f's_5_{i}'), Not(getValueFromList(V,f's_6_{i}'))))

#Each committee must be assigned a timeslot
for i in range(1,7):  
        s.append(Or (getValueFromList(V,f's_{i}_{1}'),
                     getValueFromList(V,f's_{i}_{2}'),
                     getValueFromList(V,f's_{i}_{3}'),
                     getValueFromList(V,f's_{i}_{4}'),
                     getValueFromList(V,f's_{i}_{5}'),
                     getValueFromList(V,f's_{i}_{6}')))
#print(s.assertions())

s.check()
model=s.model()
true_vars = [m for m in model if model[m]==True]
print("Model: ",true_vars)

Model:  [s_6_3, s_3_3, s_1_1, s_4_1, s_5_2, s_2_2]


## Z3 as SMT solver

Z3 can also be used to solve set of assertions using different domains. Below an example with integers, see [Z3Py Guide](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) for more.

### [Arithmetic](https://rise4fun.com/z3/tutorialcontent/guide#h24)

$$(x - y \leq 0) \land (y - z \leq 0) \land ((z - x \leq -1) \lor (z - x \leq -2))$$

In [57]:
from z3 import *

x, y, z = Ints('x y z')

s = Solver()

s.add((x - y) <= 0)
s.add((y - z) <= 0)
s.add((z - x) <= -1)
s.add((z - x) <= -2)

print(s.assertions())

if s.check() == sat:
    print(f'Solution: {s.model()}')
else:
    print("Unsatisfiable")

[x - y <= 0, y - z <= 0, z - x <= -1, z - x <= -2]
Unsatisfiable


$$(b \lor (x + y \leq 0)) \land (\neg b \lor (x + z \leq 10))$$

In [59]:
from z3 import *

x, y, z = Ints('x y z')
b = Bool('b')

s = Solver()

s.add(Or(b, (x + y) <= 0))
s.add(Or(Not(b), (x + z) <= 10))

print(s.assertions())

if s.check() == sat:
    print(f'Solution: {s.model()}')
else:
    print("Unsatisfiable")

[Or(b, x + y <= 0), Or(Not(b), x + z <= 10)]
Solution: [b = False, y = 0, z = 11, x = 0]


## Minesweeper

Let's consider the [Minesweeper](https://en.wikipedia.org/wiki/Minesweeper_(video_game)) game, where you have a grid of covered squares, some of which contain mines, but you don't know which. Your job is to uncover every square which does not contain a mine. If you uncover a square containing a mine, you lose. If you uncover a square which does not contain a mine, you are told how many mines are contained within the eight surrounding squares.

Once you start unveiling the minefield you can deduce the location of mines by solving a system of equations by trying to *disprove* that in a specific cell you can place a mine. In order to do that you can represent each cell by an integer variable that contains the number of mines in the specific cell (0 or 1).

For this exercise we will use a problem generator with the property that the generated problems can be always solved by deduction (mind that in general this is not the case). The generator is available on the [Simon Tatham's Portable Puzzle Collection](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html) page:

> The first square you open is guaranteed to be safe, and (by default) you are guaranteed to be able to solve the whole grid by deduction rather than guesswork. (Deductions may require you to think about the total number of mines.)

An interesting instance you can try to solve is the following:

[![sweeper example](/media/sweeper_6x6-10.png)](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html#6x6:2,2,m1bf215cfe)

and you can use [this link](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html#6x6:2,2,m1bf215cfe) to verify your solution. You should include below the Python code you're using to solve the instance.

A convenient representation for the problem (useful later on) is a matrix of numbers representing the number of mines *seen* by the cell or '?' for the unknown cells. E.g the above example would correspond to

    ??????
    ?1123?
    ?1003?
    ?1002?
    ?2234?
    ??????

You can use the Sudoku example in [Z3Py Guide](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) as inspiration for your code. Note that solving the game might require several calls to the solver as well as the update of the facts you know about the board when you unveil more cells.

In [154]:
# 6x6 matrix of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(6) ]
      for i in range(6) ]
bombs = [ [ Int("bombs_%s_%s" % (i+1, j+1)) for j in range(6) ]
      for i in range(6) ]


#Add initial condition of the given problem
"""
    ??????
    ?1123?
    ?1003?
    ?1002?
    ?2234?
    ??????
"""

def neighborBombs(i, j):
    rows = len(X)
    cols = len(X[0])
    count = 0

    for x in range(max(0, i - 1), min(rows, i + 2)):
        for y in range(max(0, j - 1), min(cols, j + 2)):
            count += bombs[x][y]
    return count

#si usa -1 per le celle incognite
"""
instance = ((0,2,-1,-1,-1,-1),
            (0,3,-1,-1,-1,-1),
            (2,4,-1,-1,-1,-1),
            (-1,-1,-1,-1,-1,-1),
            (-1,-1,-1,-1,-1,-1),
            (-1,-1,-1,-1,-1,-1))
"""
instance = ((-1,-1,-1,-1,-1,-1),
            (-1,1,1,2,3,-1),
            (-1,1,0,0,3,-1),
            (-1,1,0,0,2,-1),
            (-1,2,2,3,4,-1),
            (-1,-1,-1,-1,-1,-1))

#per ogni cella il numero deve rispecchiare il numero di bombe nell'intorno oppure essere bomba
minesweeper_c = [ neighborBombs(i,j) == instance[i][j]
                        for i in range(6) for j in range(6) if instance[i][j] != -1]

bombs_c = [ If(instance[i][j] == -1,
                  Or(bombs[i][j] == 0, bombs[i][j] == 1),
                  bombs[i][j] == 0)
               for i in range(6) for j in range(6) ]

number_of_bombs_c = [
    Sum([If(bombs[i][j] == 1, 1, 0) for i in range(6) for j in range(6)]) == 10
]

s = Solver()
s.add(minesweeper_c + number_of_bombs_c + bombs_c)
"""
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(bombs[i][j]) for j in range(6) ]
          for i in range(6) ]
    print_matrix(r)
"""
  
for x in range(6):
    for y in range(6):
        cell_not_safe = bombs[x][y] == 1
        s.push()
        s.add(cell_not_safe)
        if s.check() == unsat:
            print(f"Cell ({x};{y}) is safe")
        else:
            print(f"Cell ({x};{y}) is not safe")
        s.pop()


Cell (0;0) is safe
Cell (0;1) is safe
Cell (0;2) is not safe
Cell (0;3) is safe
Cell (0;4) is not safe
Cell (0;5) is safe
Cell (1;0) is safe
Cell (1;1) is safe
Cell (1;2) is safe
Cell (1;3) is safe
Cell (1;4) is safe
Cell (1;5) is not safe
Cell (2;0) is safe
Cell (2;1) is safe
Cell (2;2) is safe
Cell (2;3) is safe
Cell (2;4) is safe
Cell (2;5) is not safe
Cell (3;0) is not safe
Cell (3;1) is safe
Cell (3;2) is safe
Cell (3;3) is safe
Cell (3;4) is safe
Cell (3;5) is not safe
Cell (4;0) is safe
Cell (4;1) is safe
Cell (4;2) is safe
Cell (4;3) is safe
Cell (4;4) is safe
Cell (4;5) is safe
Cell (5;0) is safe
Cell (5;1) is safe
Cell (5;2) is not safe
Cell (5;3) is not safe
Cell (5;4) is not safe
Cell (5;5) is not safe


## Deduction in the Wumpus world

In this exercise you should consider the [Hunt the Wumpus](https://docs.google.com/document/d/1ySK0M-txOuIVWUGxPB02Ws_p5nSi5aLMlDQl4NMtAlk/edit?usp=sharing) domain, and use the solver to deduce facts about the given instance of the problem.

Use the techniques that you learned in the previous exercises to deduce (if possible) the safe cells and the position of pits and Wumpus given the perceptions that the agent would detect from a given cell.

The input is given as a matrix where cells contain either the letter `B` (breeze) or `S` (stench) according whether the agent, placed in the corresponding cell, would feel one of the two situations. We assume that the sensing information *doesn't* include the presence of pit and wumpus in the cells where they are located (see the example below for details).

E.g. a possible configuration and the corresponding "perception" matrix would be:

    +---+---+---+---+        +---+---+---+---+
    |  P| G |   |   |        |   |  B|S  |   |
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+        +---+---+---+---+
    |   |   |W  |   |        |  B|S  |   |S B|
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+ =====> +---+---+---+---+
    |   |   |   |  P|        |   |   |S B|   |
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+        +---+---+---+---+
    |   |   |   |   |        |   |   |   |  B|
    | A |   |   |   |        | A |   |   |   |
    +---+---+---+---+        +---+---+---+---+

Note that neither the cells where pits and wumpus are located don't sense breeze or stench coming from the same cell. They would sense neighbouring cells. 


Try to solve the following two examples by encoding using Z3 and explain the method you used.

1. **Example 1**
    ```
    +---+---+---+---+
    |   |  B|  B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |   |  B|  B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|   |  B|S B|
    |   |   |   |   |
    +---+---+---+---+
    |   |  B|S  |   |
    | A |   |   |   |
    +---+---+---+---+
    ```

In [117]:
from z3 import *
solver=Solver()

stench_cells = [[Bool(f's_{i}_{j}') for j in range(4)] for i in range(4)]
breeze_cells = [[Bool(f'b_{i}_{j}') for j in range(4)] for i in range(4)]
pits_cells = [[Bool(f'p_{i}_{j}') for j in range(4)] for i in range(4)]
wumpus_cells = [[Bool(f'w_{i}_{j}') for j in range(4)] for i in range(4)]
 
 
breeze_in = [(1,0),(2,0),(3,0),(1,1),(2,1),(3,1), (0,2),(2,2),(3,2),(1,3)]
stench_in = [(3,2),(2,3)]
 
#Facts
for x, y in breeze_in:
    solver.add(breeze_cells[x][y])
for x in range(4):
    for y in range(4):
        if (x,y) not in breeze_in:
            solver.add(breeze_cells[x][y] == False)
 
for x, y in stench_in:
    solver.add(stench_cells[x][y])
for x in range(4):
    for y in range(4):
        if (x,y) not in stench_in:
            solver.add(stench_cells[x][y] == False)
 
 
#Constraints
for i in range(4):
    for j in range(4):
        neighbors = [(x, y) for x, y in [(i - 1, j), (i, j - 1), (i, j + 1), (i + 1, j)] if 0 <= x < 4 and 0 <= y < 4]
        solver.add(Implies(breeze_cells[i][j], Or([pits_cells[x][y] for x, y in neighbors])))
        solver.add(Implies(stench_cells[i][j], Or([wumpus_cells[x][y] for x, y in neighbors])))
        solver.add(Implies(breeze_cells[i][j]==False, And(list([(pits_cells[x][y]==False) for x, y in neighbors]))))
        solver.add(Implies(stench_cells[i][j]==False, And(list([(wumpus_cells[x][y]==False) for x, y in neighbors]))))
 
 
if solver.check() == sat:
    model = solver.model()
    wumpus_locations = [(i, j) for i in range(4) for j in range(4) if is_true(model[wumpus_cells[i][j]])]
    pit_locations = [(i, j) for i in range(4) for j in range(4) if is_true(model[pits_cells[i][j]])]
    print("Wumpus locations:", wumpus_locations)
    print("Pit locations:", pit_locations)
else:
    print("No solution found.")

Wumpus locations: [(3, 3)]
Pit locations: [(0, 3), (2, 0), (2, 1), (3, 1)]


2. **Example 2**
    ```
    +---+---+---+---+
    |   |  B|S  |  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|S B|  B|S  |
    |   |   |   |   |
    +---+---+---+---+
    |  B|  B|S B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|  B|  B|   |
    | A |   |   |   |
    +---+---+---+---+
    ```

In [114]:
from z3 import *
solver=Solver()
 
 
stench_cells = [[Bool(f's_{i}_{j}') for j in range(4)] for i in range(4)]
breeze_cells = [[Bool(f'b_{i}_{j}') for j in range(4)] for i in range(4)]
pits_cells = [[Bool(f'p_{i}_{j}') for j in range(4)] for i in range(4)]
wumpus_cells = [[Bool(f'w_{i}_{j}') for j in range(4)] for i in range(4)]
 
 
breeze_in = [(1,0),(3,0),(0,1),(1,1),(2,1),(0,2),(1,2),(2,2),(3,2),(0,3),(1,3),(2,3)]
stench_in = [(2,0),(1,1), (3,1), (2,2)]
 
#Facts
for x, y in breeze_in:
    solver.add(breeze_cells[x][y])
for x in range(4):
    for y in range(4):
        if (x,y) not in breeze_in:
            solver.add(breeze_cells[x][y] == False)
 
for x, y in stench_in:
    solver.add(stench_cells[x][y])
for x in range(4):
    for y in range(4):
        if (x,y) not in stench_in:
            solver.add(stench_cells[x][y] == False)
 
 
#Constraints
for i in range(4):
    for j in range(4):
        neighbors = [(x, y) for x, y in [(i - 1, j), (i, j - 1), (i, j + 1), (i + 1, j)] if 0 <= x < 4 and 0 <= y < 4]
        print([pits_cells[x][y] for x, y in neighbors])
        solver.add(Implies(breeze_cells[i][j], Or([pits_cells[x][y] for x, y in neighbors])))
        solver.add(Implies(stench_cells[i][j], Or([wumpus_cells[x][y] for x, y in neighbors])))
        solver.add(Implies(breeze_cells[i][j]==False, And(list([(pits_cells[x][y]==False) for x, y in neighbors]))))
        solver.add(Implies(stench_cells[i][j]==False, And(list([(wumpus_cells[x][y]==False) for x, y in neighbors]))))
for i in range(4) :
    print([breeze_cells[i][j] for j in range(4)])
     
 
if solver.check() == sat:
    model = solver.model()
    wumpus_locations = [(i, j) for i in range(4) for j in range(4) if is_true(model[wumpus_cells[i][j]])]
    pit_locations = [(i, j) for i in range(4) for j in range(4) if is_true(model[pits_cells[i][j]])]
    print("Wumpus locations:", wumpus_locations)
    print("Pit locations:", pit_locations)
else:
    print("No solution found.")

[p_0_1, p_1_0]
[p_0_0, p_0_2, p_1_1]
[p_0_1, p_0_3, p_1_2]
[p_0_2, p_1_3]
[p_0_0, p_1_1, p_2_0]
[p_0_1, p_1_0, p_1_2, p_2_1]
[p_0_2, p_1_1, p_1_3, p_2_2]
[p_0_3, p_1_2, p_2_3]
[p_1_0, p_2_1, p_3_0]
[p_1_1, p_2_0, p_2_2, p_3_1]
[p_1_2, p_2_1, p_2_3, p_3_2]
[p_1_3, p_2_2, p_3_3]
[p_2_0, p_3_1]
[p_2_1, p_3_0, p_3_2]
[p_2_2, p_3_1, p_3_3]
[p_2_3, p_3_2]
[b_0_0, b_0_1, b_0_2, b_0_3]
[b_1_0, b_1_1, b_1_2, b_1_3]
[b_2_0, b_2_1, b_2_2, b_2_3]
[b_3_0, b_3_1, b_3_2, b_3_3]
Wumpus locations: [(2, 1)]
Pit locations: [(0, 2), (1, 2), (2, 0), (2, 2)]


Now you should consider the case that only partial information is available, i.e. for some cells you don't have any information (below they're marked with `?`). This means that you can have several possible configurations (i.e. *models* in logic terms) corresponding to the available informations.

You should encode the configuration below (it's the first example with some unknown cells) and verify whether the first model the solver returns is the same as the one found for the first example.

```
+---+---+---+---+
|   |  B|  ?|  ?|
|   |   |   |   |
+---+---+---+---+
|   |  B|  ?|  B|
|   |   |   |   |
+---+---+---+---+
|  B|  ?|  ?|  ?|
|   |   |   |   |
+---+---+---+---+
|   |  B|S  |   |
| A |   |   |   |
+---+---+---+---+
```


In [132]:
from z3 import *
solver=Solver()

stench_cells = [[Bool(f's_{i}_{j}') for j in range(4)] for i in range(4)]
breeze_cells = [[Bool(f'b_{i}_{j}') for j in range(4)] for i in range(4)]
pits_cells = [[Bool(f'p_{i}_{j}') for j in range(4)] for i in range(4)]
wumpus_cells = [[Bool(f'w_{i}_{j}') for j in range(4)] for i in range(4)]
 
 
breeze_in = [(1,0),(1,1),(3,1), (0,2),(1,3)]
stench_in = [(2,3)]
unknowns = [(2,0),(3,0),(2,1),(1,2),(2,2),(3,2)]
 
#Facts
for x, y in breeze_in:
    solver.add(breeze_cells[x][y])
for x in range(4):
    for y in range(4):
        if (x,y) not in breeze_in and (x,y) not in unknowns:
            solver.add(breeze_cells[x][y] == False)
 
for x, y in stench_in:
    solver.add(stench_cells[x][y])
for x in range(4):
    for y in range(4):
        if (x,y) not in stench_in and (x,y) not in unknowns:
            solver.add(stench_cells[x][y] == False)
 
 
#Constraints
for i in range(4):
    for j in range(4):
        neighbors = [(x, y) for x, y in [(i - 1, j), (i, j - 1), (i, j + 1), (i + 1, j)] if 0 <= x < 4 and 0 <= y < 4]
        solver.add(Implies(breeze_cells[i][j], Or([pits_cells[x][y] for x, y in neighbors])))
        solver.add(Implies(stench_cells[i][j], Or([wumpus_cells[x][y] for x, y in neighbors])))
        solver.add(Implies(breeze_cells[i][j]==False, And(list([(pits_cells[x][y]==False) for x, y in neighbors]))))
        solver.add(Implies(stench_cells[i][j]==False, And(list([(wumpus_cells[x][y]==False) for x, y in neighbors]))))
 
 
if solver.check() == sat:
    model = solver.model()
    print(model)
    wumpus_locations = [(i, j) for i in range(4) for j in range(4) if is_true(model[wumpus_cells[i][j]])]
    pit_locations = [(i, j) for i in range(4) for j in range(4) if is_true(model[pits_cells[i][j]])]
    print("Wumpus locations:", wumpus_locations)
    print("Pit locations:", pit_locations)
else:
    print("No solution found.")

[b_3_0 = True,
 w_2_0 = False,
 w_0_3 = False,
 s_0_0 = False,
 b_2_0 = True,
 s_1_2 = True,
 s_2_3 = True,
 s_1_0 = False,
 s_3_2 = True,
 b_2_1 = True,
 b_0_2 = True,
 p_1_2 = True,
 b_1_2 = False,
 w_0_2 = False,
 p_1_3 = False,
 s_3_0 = False,
 w_1_0 = False,
 s_0_2 = False,
 w_1_3 = False,
 w_2_1 = False,
 w_3_0 = False,
 w_3_2 = False,
 p_1_1 = False,
 b_1_0 = True,
 w_0_1 = False,
 p_2_2 = False,
 p_0_0 = False,
 w_2_2 = True,
 b_0_1 = False,
 p_3_2 = False,
 b_3_3 = False,
 p_3_3 = False,
 b_1_1 = True,
 b_3_1 = True,
 s_0_3 = False,
 s_2_0 = False,
 p_3_0 = True,
 b_0_3 = False,
 b_2_3 = False,
 w_0_0 = False,
 p_3_1 = False,
 s_3_3 = False,
 w_3_1 = False,
 s_1_1 = False,
 s_1_3 = False,
 w_1_2 = False,
 b_1_3 = True,
 b_3_2 = False,
 p_0_3 = False,
 s_3_1 = False,
 b_2_2 = True,
 p_1_0 = False,
 p_0_2 = False,
 w_1_1 = False,
 w_2_3 = False,
 p_0_1 = False,
 w_3_3 = False,
 p_2_1 = False,
 s_0_1 = False,
 b_0_0 = False,
 p_2_3 = False,
 p_2_0 = True,
 s_2_1 = True,
 s_2_2 = 

## Example: enumerating all models

Using the API you can write a procedure to enumerate all the models. The key to avoid the repetition of models that have been already enumerated is to add a formulae that falsify the models already discovered in such a way that they would be avoided in the search.

Consider that a model is a set of variable assignments; therefore a formula that is blocking the model is the negation of the value assignments; e.g.:

In [133]:
from z3 import *
Tie, Shirt = Bools('Tie Shirt')
s = Solver()
s.add(Xor(Tie, Shirt))

while s.check() == sat:
    print(s.model())
    s.add(Or([v() != s.model()[v] for v in s.model()]))

[Tie = True, Shirt = False]
[Tie = False, Shirt = True]


Note the `v()` reference to the variable from the model and be aware that the above technique doesn't work with functions and arrays, see [this](https://stackoverflow.com/a/11869410) stackoverflow answer for details. A more advanced technique is described in [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-blocking-evaluations).